In [45]:
!pip install datasets

In [46]:
from datasets import load_dataset, DatasetDict

In [47]:
# KoAlpaca v1.1 데이터셋 -> 네이버 지식인 크롤링
koalpaca_dataset = load_dataset("beomi/KoAlpaca-v1.1a")

In [48]:
koalpaca_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [49]:
koalpaca_dataset["train"][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [50]:
koalpaca_dataset['train'].features.keys()

dict_keys(['instruction', 'output', 'url'])

In [51]:
def kolapaca_data_to_instruction(d):
    d['text'] = f"### User\n{d['instruction']}\n\n### Bot\n{d['output']}"
    return d

instruction_koalpaca_dataset = koalpaca_dataset.map(kolapaca_data_to_instruction, remove_columns=["instruction", "output",'url'])

In [52]:
print(instruction_koalpaca_dataset)
instruction_koalpaca_dataset['train'][0]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 21155
    })
})


{'text': '### User\n양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### Bot\n양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.'}

In [53]:
# ShareGPT data
# Reference: https://huggingface.co/datasets/junelee/sharegpt_deepl_ko
# ShareGPT의 데이터를 DeepL로 번역한 데이터

sharegpt_dataset = load_dataset("junelee/sharegpt_deepl_ko", data_files="ko_dataset_2.json")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [54]:
print(sharegpt_dataset)
print('----------------------------')
sharegpt_dataset['train'][0]['conversations'][:2]

DatasetDict({
    train: Dataset({
        features: ['conversations', 'id'],
        num_rows: 84416
    })
})
----------------------------


[{'from': 'human',
  'markdown': None,
  'text': None,
  'value': '고객을 위해 이러한 전략과 전술을 구현하는 성장 마케팅 대행사와 관련하여 Jeff Walker의 제품 출시 공식의 주요 아이디어를 글머리 기호로 요약하세요...'},
 {'from': 'gpt',
  'markdown': None,
  'text': None,
  'value': '다음은 성장 마케팅 대행사가 고객을 위해 적용할 수 있는 제프 워커의 제품 출시 공식의 주요 아이디어입니다:\r\n\r\n1. 타겟 고객과 그들의 니즈를 파악합니다: 제품이나 서비스의 이상적인 고객을 파악하고 이들에게 공감을 불러일으킬 수 있는 메시지를 작성합니다.\r\n2. 사전 출시: 입소문을 만들고, 사용 후기 및 사례 연구를 수집하고, 소셜 미디어를 사용하여 인지도를 높여 출시에 대한 기대감과 흥미를 조성합니다.\r\n3. 출시: 잘 짜여진 출시 시퀀스를 사용하여 판매와 전환을 극대화하세요. 여기에는 보너스 제공, 희소성 창출, 마감일을 사용하여 긴박감을 조성하는 것이 포함될 수 있습니다.\r\n4. 출시 후: 고객과 후속 조치를 취하고, 피드백을 수집하고, 지속적으로 가치를 제공하여 고객의 참여와 충성도를 유지하세요.\r\n5. 제품군을 만듭니다: 초기 제품이 성공하면 동일한 고객의 다른 요구 사항을 해결하는 추가 제품 또는 서비스를 만들어 제품을 확장합니다.\r\n6. 지속적인 최적화: 출시 프로세스와 제품군을 지속적으로 모니터링하고 최적화하여 결과를 개선합니다.\r\n7. 커뮤니티 구축: 출시 프로세스를 사용하여 제품에 대한 열정이 있고 널리 알리는 데 도움을 줄 수 있는 고객 커뮤니티를 구축하세요.\r\n8. 자동화를 활용하세요: 기술 및 자동화를 사용하여 출시 프로세스를 간소화하고 효율성을 개선하세요.'}]

In [55]:
# 주거니 받거니 하는 데이터를 하나의 텍스트 덩어리로 합치기

def sharegpt_data_to_instruction(data):
    text = []
    for i in data['conversations']:
        if i['from'] == 'human':
            text.append(f"### User\n{i['value']}")
        else:
            text.append(f"### Bot\n{i['value']}")
    data['text'] = '\n\n'.join(text)
    return data


instruction_sharegpt_dataset = sharegpt_dataset.map(sharegpt_data_to_instruction, remove_columns=["conversations",'id'])

In [56]:
# 두 데이터 합치기
from datasets import concatenate_datasets

instruction_dataset = concatenate_datasets([instruction_koalpaca_dataset['train'], instruction_sharegpt_dataset['train']],axis=0)

In [57]:
instruction_dataset

Dataset({
    features: ['text'],
    num_rows: 105571
})

In [58]:
# Train Valid Test 나누기
train_validtest = instruction_dataset.train_test_split(test_size=0.2, seed=42)
valid_test = train_validtest['test'].train_test_split(test_size=0.5, seed=42)
instruction_dataset = DatasetDict({
    'train': train_validtest['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [59]:
instruction_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 84456
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 10557
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10558
    })
})

In [60]:
# 저장하기
instruction_dataset.save_to_disk("./instruction_dataset")

Saving the dataset (0/2 shards):   0%|          | 0/84456 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10557 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10558 [00:00<?, ? examples/s]